## Agente LangChain — Clima em Tempo Real

Este notebook implementa um agente que consulta o clima atual de qualquer lugar do mundo usando LLM no OCI + API Open-Meteo.

### O agente decide sozinho:

- Latitude
- Longitude
- Quando chamar a API de clima

### Funcionalidades

- Consulta clima em tempo real (Open-Meteo)
- Agente com autonomia de coordenadas
- Processamento com LLM via OCI
- Busca inteligente com LangChain (ReAct)

### Passo 1 - Instalando dependencias

In [2]:
!python -m pip install langchain-oci langchain-core langchain==1.1.0 
!python -m pip install requests

  Using cached langchain_core-1.2.14-py3-none-any.whl.metadata (4.4 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached langgraph_checkpoint-4.0.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached langgraph_prebuilt-1.0.8-py3-none-any.whl.metadata (5.2 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached anyio-4.12.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.5-cp314-cp314-win_amd64.whl.metadata (7.4 kB)
  Using cached t

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Passo 2 - Tool de Clima (Open-Meteo)

O agente só precisa fornecer latitude e longitude.

In [3]:
import requests
from langchain_core.tools import tool

@tool
def get_weather(latitude: str, longitude: str) -> str:
    """Obtém o clima atual usando latitude e longitude."""
    
    url = (
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m,wind_speed_10m"
    )

    response = requests.get(url)
    data = response.json()
    
    return str(data)


c:\Users\Amanda Machado\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


### Passo 3 - Configuração do Agente

In [4]:
import json
from langchain_core.callbacks import BaseCallbackHandler

class CleanAgentCallback(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, **kwargs):
        tool_name = serialized.get("name", "unknown_tool")
        print(f"\n TOOL: {tool_name}")
        print(f"   ↳ input: {input_str}")

    def on_tool_end(self, output, **kwargs):
        print("TOOL RESULT (resumo):")

        content = getattr(output, "content", output)

        try:
            data = json.loads(content)

            if "current" in data:
                temp = data["current"].get("temperature_2m")
                wind = data["current"].get("wind_speed_10m")

                print(f"   ↳ Temperatura: {temp}°C")
                print(f"   ↳ Vento: {wind} km/h")

        except Exception:
            text = str(content)
            print("   ↳", text[:200])


### Passo 4 - LLM

In [5]:
from langchain_oci.chat_models import ChatOCIGenAI

llm = ChatOCIGenAI(
  model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyajqi26fkxly6qje5ysvezzrypapl7ujdnqfjq6hzo2loq",
  service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
  compartment_id="ocid1.compartment.oc1..aaaaaaaa3x7n7wwfnghe4imvt3niwo76wgqv6ecn2iadiwoph73jjowbhbna",
  provider="meta",
  model_kwargs={
    "temperature": 0.3,
    "max_tokens": 800,
    "top_p": 0.8,
  },
  auth_type="API_KEY",
  auth_profile="CHICAGOV2"
)


### Passo 5 - Lista de Tools

In [6]:
tools = [
    get_weather
]


### Passo 6 - Prompt ReAct

Aqui instruímos o agente a:

- Escolher latitude/longitude
- Analisar o clima
- Dar insights úteis

In [7]:
react_prompt_template = """You are a helpful weather assistant.

You can choose any latitude and longitude needed to answer.

IMPORTANT:
- Always choose coordinates yourself
- After using the tool, analyze the data
- Do not output raw JSON
- Provide insights

{tools}

Use EXACTLY this format:

Question: the input question
Thought: reasoning
Action: tool name
Action Input: latitude,longitude
Observation: tool result
Thought: analysis
Final Answer: helpful answer in Brazilian Portuguese

Begin!
"""


### Passo 7 - Criação do Agente

In [9]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=react_prompt_template
)


### Passo 8 - Função de Extração da Resposta Final

In [10]:
from langchain_core.messages import AIMessage

def extract_final_output(agent_response):
    final_text = None
    total_tokens = None

    for msg in reversed(agent_response["messages"]):
        if isinstance(msg, AIMessage):

            if msg.content and final_text is None:
                if "Final Answer:" in msg.content:
                    final_text = msg.content.split("Final Answer:",1)[1].strip()
                else:
                    final_text = msg.content.strip()

            if total_tokens is None:
                total_tokens = (
                    msg.response_metadata.get("total_tokens")
                    or msg.additional_kwargs.get("total_tokens")
                )

        if final_text and total_tokens:
            break

    return final_text, total_tokens


### Teste 1 — Clima em Paris

In [11]:
callback = CleanAgentCallback()

pergunta = "Como está o clima agora em Paris?"

response = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)

final_answer, total_tokens = extract_final_output(response)

print("\nFinal answer:")
print(final_answer)

print("\nMetadados:")
print(f"total tokens: {total_tokens}")



 TOOL: get_weather
   ↳ input: {'latitude': '48.8567', 'longitude': '2.3508'}
TOOL RESULT (resumo):
   ↳ {'latitude': 48.86, 'longitude': 2.3599997, 'generationtime_ms': 0.05626678466796875, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 45.0, 'current_units': {'

Final answer:
O clima em Paris está relativamente ameno, com uma temperatura de 11.6°C e ventos moderados de 14.2 km/h.

Metadados:
total tokens: 679


### Teste 2 — Clima no Rio

In [12]:
pergunta = "Está calor no Rio de Janeiro agora?"

response = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)

final_answer, total_tokens = extract_final_output(response)

print("\nFinal answer:")
print(final_answer)

print("\nMetadados:")
print(f"total tokens: {total_tokens}")



 TOOL: get_weather
   ↳ input: {'latitude': '-22.9068', 'longitude': '-43.1729'}
TOOL RESULT (resumo):
   ↳ {'latitude': -22.875, 'longitude': -43.25, 'generationtime_ms': 0.05042552947998047, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 12.0, 'current_units': {'t

Final answer:
Sim, está calor no Rio de Janeiro agora, com uma temperatura de 25.3°C.

Metadados:
total tokens: 651
